Connection

In [5]:
import urllib3
urllib3.disable_warnings()  # Suppress self-signed cert warnings

from requests.auth import HTTPBasicAuth
import requests
import pprint as pp

host = 'localhost'
port = 9200
user = 'admin'
password = 'MyStr0ng@Pass'
index_name = user

url = f"https://{host}:{port}"

try:
    res = requests.get(url, auth=HTTPBasicAuth(user, password), verify=False)
    pp.pprint(res.json())
except requests.exceptions.RequestException as e:
    print("Request failed:", e)


{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'L0dPWzM1RbiGcZAnblOjaA',
 'name': 'fea6a945eb61',
 'tagline': 'The OpenSearch Project: https://opensearch.org/',
 'version': {'build_date': '2025-02-27T01:16:47.726162386Z',
             'build_hash': '2e4741fb45d1b150aaeeadf66d41445b23ff5982',
             'build_snapshot': False,
             'build_type': 'tar',
             'distribution': 'opensearch',
             'lucene_version': '9.12.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.10.0',
             'number': '2.19.1'}}


Load Captions

In [6]:
import pandas as pd
from scripts.parse_activitynet import load_captions

# Load all captions
data = load_captions(r"C:\Users\nunoj\Desktop\MPWD\Project\VideoDialog\captions\train.json")
df = pd.DataFrame(data)

# Step 1: Find videos that mention "car"
contains_car = df[df['caption'].str.contains(r'\bcar\b', case=False, na=False)]
car_video_ids = set(contains_car['video_id'])

# Step 2: Get videos with more than 5 captions
caption_counts = df['video_id'].value_counts()
videos_with_5plus = set(caption_counts[caption_counts > 8].index)

# Step 3: Intersection → videos that meet both criteria
valid_video_ids = list(car_video_ids & videos_with_5plus)

# Step 4: Pick just 5 videos from the intersection
five_video_ids = valid_video_ids[:5]  # you can randomize with `random.sample` if needed

# Step 5: Get all captions for those 5 videos
selected_df = df[df['video_id'].isin(five_video_ids)]

# Confirm selection
print(f"Selected {len(selected_df)} captions from 5 videos:")
print(selected_df.groupby("video_id").size())


Selected 49 captions from 5 videos:
video_id
v_06ofnvq2Hjs    10
v_Eilil6FZhK8     9
v_Gms3Yt6RrV4     9
v_MWdPh6J-YXM    10
v_S5kuckj4Ud4    11
dtype: int64


Opening Index

In [8]:
import pprint as pp
from opensearchpy import OpenSearch
from opensearchpy import helpers

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    use_ssl = True,
    #url_prefix = 'opensearch_v2',
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

if client.indices.exists(index_name):

    resp = client.indices.open(index = index_name)
    print(resp)

    print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
    settings = client.indices.get_settings(index = index_name)
    pp.pprint(settings)

    print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
    mappings = client.indices.get_mapping(index = index_name)
    pp.pprint(mappings)

    print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
    print(client.count(index = index_name))
else:
    print("Index does not exist.")


{'acknowledged': True, 'shards_acknowledged': True}

----------------------------------------------------------------------------------- INDEX SETTINGS
{'admin': {'settings': {'index': {'creation_date': '1744656979538',
                                  'knn': 'true',
                                  'number_of_replicas': '0',
                                  'number_of_shards': '1',
                                  'provided_name': 'admin',
                                  'refresh_interval': '1s',
                                  'replication': {'type': 'DOCUMENT'},
                                  'uuid': 'yKKN0QCUTwy--jKn2IBSTg',
                                  'version': {'created': '136407927'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'admin': {'mappings': {'dynamic': 'strict',
                        'properties': {'caption': {'analyzer': 'standard',
                                                   'type':

Close Index

In [ ]:
resp = client.indices.close(index = index_name)
print(resp)

In [7]:
index_body = {
    "settings": {
        "index": {
            "number_of_replicas": 0,
            "number_of_shards": 1,
            "refresh_interval": "1s",
            "knn": "true"
        }
    },
    "mappings": {
        "dynamic": "strict",
        "properties": {
            "video_id": {"type": "keyword"},
            "start": {"type": "float"},
            "end": {"type": "float"},
            "duration": {"type": "float"},
            "caption": {"type": "text", "analyzer": "standard"},
            "video_url": {"type": "keyword"},
            # For after generating embeddings
            # "caption_vec": {
            #     "type": "knn_vector",
            #     "dimension": 512,  # whatever the model uses
            #     "method": {
            #         "name": "hnsw",
            #         "engine": "nmslib",
            #         "space_type": "cosinesimil"
            #     }
            # }
        }
    }
}


if client.indices.exists(index=index_name):
    print("Index already existed. Nothing to be done.")
else:        
    response = client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)


NameError: name 'client' is not defined

Check the index

In [9]:
print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
index_settings = {
    "settings":{
      "index":{
         "refresh_interval" : "1s"
      }
   }
}
pp.pprint(client.indices.get_alias(index_name))

client.indices.put_settings(index = index_name, body = index_settings)
settings = client.indices.get_settings(index = index_name)
pp.pprint(settings)

print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
mappings = client.indices.get_mapping(index = index_name)
pp.pprint(mappings)

print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
print(client.count(index = index_name))


----------------------------------------------------------------------------------- INDEX SETTINGS
{'admin': {'aliases': {}}}
{'admin': {'settings': {'index': {'creation_date': '1744656979538',
                                  'knn': 'true',
                                  'number_of_replicas': '0',
                                  'number_of_shards': '1',
                                  'provided_name': 'admin',
                                  'refresh_interval': '1s',
                                  'replication': {'type': 'DOCUMENT'},
                                  'uuid': 'yKKN0QCUTwy--jKn2IBSTg',
                                  'version': {'created': '136407927'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'admin': {'mappings': {'dynamic': 'strict',
                        'properties': {'caption': {'analyzer': 'standard',
                                                   'type': 'text'},
               

 Index deletion

In [ ]:
This line is here to prevent you from inadvertently deleting data.

if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name
    )
    print('\nDeleting index:')
    print(response)

Built-in document tokenizers and analyzers

In [18]:
anls = {
  "analyzer": "whitespace",
  "text": "The car speeds down the highway"
}
client.indices.analyze(body=anls, index=index_name)

{'tokens': [{'token': 'The',
   'start_offset': 0,
   'end_offset': 3,
   'type': 'word',
   'position': 0},
  {'token': 'car',
   'start_offset': 4,
   'end_offset': 7,
   'type': 'word',
   'position': 1},
  {'token': 'speeds',
   'start_offset': 8,
   'end_offset': 14,
   'type': 'word',
   'position': 2},
  {'token': 'down',
   'start_offset': 15,
   'end_offset': 19,
   'type': 'word',
   'position': 3},
  {'token': 'the',
   'start_offset': 20,
   'end_offset': 23,
   'type': 'word',
   'position': 4},
  {'token': 'highway',
   'start_offset': 24,
   'end_offset': 31,
   'type': 'word',
   'position': 5}]}

In [ ]:
anls = {
  "analyzer": "standard",
  "text": "The car speeds down the highway"
}
client.indices.analyze(body=anls, index=index_name)


In [19]:
for idx, row in selected_df.iterrows():
    moment = {
        'video_id': row['video_id'],
        'caption': row['caption'],
        'start': row['start'],
        'end': row['end'],
        'duration': row['duration'],
        'video_url': row['video_url'],
    }
    
    response = client.index(index=index_name, id=f"{row['video_id']}_{row['start']}", body=moment)
    print(response['result'])


updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated


## Deleting a single document

In [ ]:
This line is here to prevent you from inadvertently deleting data.

response = client.delete(
    index = index_name,
    id = id
)

print('\nDeleting document:')
print(response)

Text-based search

In [21]:
qtxt = "fast car"

query_bm25 = {
  'size': 5,
  '_source': ['caption', 'video_id', 'start', 'end', 'video_url'],
  'query': {
    'multi_match': {
      'query': qtxt,
      'fields': ['caption']
    }
  }
}

response = client.search(
    body=query_bm25,
    index=index_name
)

print('\nSearch results:')
for hit in response['hits']['hits']:
    source = hit['_source']
    print(f"- {source['caption']} (video: {source['video_id']}, time: {source['start']}s → {source['end']}s)")



Search results:
-  The man puts the spare tire in the car and gets in the car. (video: v_S5kuckj4Ud4, time: 172.53s → 195.53s)
-  A man cleans snow off a car. (video: v_9V7cMp_w1_0, time: 75.06s → 94.49s)
-  A car drives away from the camera. (video: v_9V7cMp_w1_0, time: 147.47s → 150.12s)
-  We see a cop car on the street. (video: v_06ofnvq2Hjs, time: 92.59s → 96.66s)
-  The camera shows several men in a car. (video: v_MWdPh6J-YXM, time: 96.04s → 96.9s)


Term-based search

In [35]:
query_bm25 = {
  'size':10,
  '_source': ['caption'],
  'query': {
        "term": {
            "video_id" : 'v_06ofnvq2Hjs'
        }
   }
}

response = client.search(
    body = query_bm25,
    index = index_name
)

print('\nSearch results:')
for hit in response['hits']['hits']:
    source = hit['_source']
    print(f"- {source['caption']}")



Search results:
-  A lady holds up a USA Flag and talks.
-  We see a cop car on the street.
-  We see Anthony Shetler sitting and talking before skating on the street.
-  A man jumps stairs and falls.
-  We see the ending credits for the video.
- We see images which were  shot around DC.
-  We see a man talking and shots of skateboards as people assemble in a room.
-  We see Tony Hawk who points to other people.
-  We see people skateboarding in the street, and around DC.
-  We see a skater named Shaun Gregoirie.


Bool-based search

In [37]:
query_bm25 = {
  'size': 10,
  '_source': ['caption'],
  'query': {
    'bool': {
      'must': [
        { 'term': { 'video_id': 'v_06ofnvq2Hjs' } }
      ],
      'should': [
        { 'match': { 'caption': 'car' } }
      ]
    }
  }
}

response = client.search(
    body=query_bm25,
    index=index_name
)

print('\nSearch results:')
for hit in response['hits']['hits']:
    source = hit['_source']
    print(f"- {source['caption']}")



Search results:
-  We see a cop car on the street.
-  A lady holds up a USA Flag and talks.
-  We see Anthony Shetler sitting and talking before skating on the street.
-  A man jumps stairs and falls.
-  We see the ending credits for the video.
- We see images which were  shot around DC.
-  We see a man talking and shots of skateboards as people assemble in a room.
-  We see Tony Hawk who points to other people.
-  We see people skateboarding in the street, and around DC.
-  We see a skater named Shaun Gregoirie.
